In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
ipy = get_ipython()
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
import re
import numpy as np

scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))

import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl

def mplresestcol():
    COLOR = 'black'
    COLOR_INV = 'white'
    mpl.rcParams['text.color'] = COLOR
    mpl.rcParams['axes.labelcolor'] = COLOR
    mpl.rcParams['xtick.color'] = COLOR
    mpl.rcParams['ytick.color'] = COLOR
    mpl.rcParams["figure.facecolor"] = COLOR_INV
    mpl.rcParams["axes.facecolor"] = COLOR_INV

mplresestcol()

In [ ]:
subj = 'pilot_Dmitrii_20230426'

from glob import glob
from pathlib import Path
dmeg = '/home/demitau/data_Quentin/pilot_studies/context_change_MEG/'
fnbs = []
subjects = []
subj2path = {}
subj2dsnames = {}
for fnf in os.scandir(dmeg):
    if fnf.is_dir():
        print(fnf, fnf.name)
        subjects += [fnf.name]
        subj2path[fnf.name] = fnf.path
        
        subj2dsnames[fnf.name] = []
        
        for fnf2 in os.scandir(fnf.path):
            #print(fnf2.name)
            if fnf2.name.endswith('.ds'):
                subj2dsnames[fnf.name] += [fnf2.path]
        subj2dsnames[fnf.name] = list(sorted(subj2dsnames[fnf.name]))
    

p = pjoin(subj2path[subj], 'preproc')
#fnf = pjoin(p, f'decres_naive_{ph}_{vn}_{tmin:.2f}_{tmax:.2f}_{X.shape}_flt{freqs_flt}' )      

fns = os.listdir(p)                                                                                                                                 
 # Loop through the list of files and extract the parts using regex groups                                                                           
pattern = (r"decres_naive_([A-Z_]+)_([a-z0-9_]+)_([0-9\-]+\.\d+)_([\d\-]+\.\d+)_(\(\d+, \d+, \d+\))"                                                       
        r"_flt\((.+)\).npz")

res = []
# Loop through the list of files and extract the parts using regex groups                                                                               
for fn in fns:     
    if not isinstance(fn,str):
        continue
    match = re.search(pattern, fn)   
    #if fn.startswith('decres'):
    #    print(fn)
    if not match:                                                                                                                                   
        continue                                                                                                                                    

    f = np.load(pjoin(p,fn))
    scores = f['scores']
    scores_perm = f['scores_perm']
        
    ph = match.group(1) # The first group is the phase                                                                                           
    vn = match.group(2) # The second group is the varn                                                                                            
    tmin = float( match.group(3) ) # The third group is the tmin                                                                                             
    tmax = float( match.group(4) ) # The fourth group is the tmax    
    tlim = (tmin,tmax)
    Xshape = match.group(5) # The fifth group is the Xshape      
    freqs  = match.group(6) # The fifth group is the Xshape      
    
    td = None

    print(match, Xshape, freqs, scores_perm.shape)
    
    scores_mean = scores.mean(0)
    if scores_perm.ndim == 2:
        scores_perm_mean = scores_perm.mean(0)
    else:
        scores_perm_mean = [None] * len(scores_mean)
    
    
    #dt = (float(tmax) - float(tmin)) / eval(Xshape)[-2]
    if 'times' in list(f.keys()):
        ts = f['times'][::f['tskip'] ]
    else:
        ts = np.linspace( float(tmin), float(tmax), len(scores_mean))
    assert len(ts) == len(scores_mean), (len(ts), len(scores_mean))
    #ts
    print(list(f.keys()))
    
    r = [(t, ph, vn, sm, smp, Xshape, freqs, *tlim, td) for t,sm,smp in zip(ts,scores_mean, scores_perm_mean)]
    dftmp = pd.DataFrame(r, columns=['t','phase','varn','score','score_perm', 
                                     'Xshape','freqs','tmin', 'tmax', 'calc_dur'])               
    
    auxpars = ['nperm','nbfold','nbfolds_perm','decim','sfreq','tskip','n_jobs','td']
    for ap in auxpars:
        if ap in list( f.keys() ):
            dftmp[ap] = f[ap]
    
    res += [dftmp]

In [ ]:
import pandas as pd
nperm = 6
nbfold = 5

#dfdecres['tmin'] = dfdecres['tlim'].apply(lambda x: x[0]) 
dfdecres = pd.concat(res,ignore_index=True)
dfdecres

In [ ]:
import seaborn as sns
fg= sns.relplot(data=dfdecres, kind='line', x='t', y='score', hue='varn', col='phase')
for ax in fg.axes.flatten():
    std = dfdecres['score_perm'].std()
    ax.axhline(y=std , color='r', ls=':')
    ax.axhline(y=-std , color='r', ls=':')
#sns.relplot(data=dfdecres, kind='line', x='t', y='score_perm', hue='varn', col='phase', ax=fg.axes[0])

In [ ]:
fnp = 'pilot_Dmitrii_context_change_20230426_124646.param'
path_behav = '/home/demitau/data_Quentin/pilot_studies/context_change_MEG/pilot_Dmitrii_20230426/behav'
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
params, phase2trigger, trigger2phase, CONTEXT_TRIGGER_DICT  = readParamFiles(fnp, path_behav)

ITI_duration = float(params["ITI_duration"])
motor_prep_duration = float(params["motor_prep_duration"])
time_feedback = float(params["time_feedback"])
time_at_home = float(params["time_at_home"])
perf_feedback_duration = float(params["perf_feedback_duration"])
perf_notif_start_delay = float(params["perf_notif_start_delay"])

print("ITI_duration:", ITI_duration)
print("motor_prep_duration:", motor_prep_duration)
print("time_feedback:", time_feedback)
print("time_at_home:", time_at_home)
print("perf_feedback_duration:", perf_feedback_duration)
print("perf_notif_start_delay:", perf_notif_start_delay)
# REST, GOCUE, MVT, ITI

In [ ]:
# This it is not really true in general beacause this can vary depending on what participant does
ph2phst_rel_trialst = {'REST':0, 'GO_CUE_WAIT_AND_SHOW':time_at_home, 
                       'TARGET_AND_FEEDBACK':time_at_home+motor_prep_duration,
                      'ITI':time_at_home+motor_prep_duration+time_feedback}
ph2phst_rel_ITIst = {'REST':-time_feedback-motor_prep_duration-time_at_home, 
                     'GO_CUE_WAIT_AND_SHOW':-time_feedback-motor_prep_duration, 
                       'TARGET_AND_FEEDBACK':-time_feedback,
                      'ITI':0, 'next_REST':ITI_duration,  
                     'next_GO_CUE_WAIT_AND_SHOW':ITI_duration+time_at_home,
                    'next_TARGET_AND_FEEDBACK':ITI_duration+time_at_home+motor_prep_duration,
                    'next_ITI': ITI_duration+time_at_home+motor_prep_duration + time_feedback}

#ph2offest = {''}
#vlines_per_ph = {'ITI': }

In [ ]:
dfdecres.groupby('varn')['score_perm'].std()

In [ ]:
ylim = -0.15,0.3

In [ ]:
plt.subplots_adjust?

In [ ]:
import seaborn as sns
#dfdecres.query('varn != "tgti_to_show"')
fg= sns.relplot(data=dfdecres, kind='line', 
                x='t', y='score', #hue='varn', 
                col='phase', row='varn', aspect=1.5)
#fg.set(xticks=np.arange(-2,6,1))
for ax in fg.axes.flatten():
    std = dfdecres['score_perm'].std()
    ax.axhline(y=std , color='r', ls=':')
    ax.axhline(y=-std , color='r', ls=':')
    ax.vlines(ph2phst_rel_ITIst.values(), *ylim, ls=':', color='k')
    ax.grid(True, color='lightgrey')
    #ax.set_xticks(np.arange(-2,6,1))
#plt.subplots_adjust(sharex='no')
#sns.relplot(data=dfdecres, kind='line', x='t', y='score_perm', hue='varn', col='phase')

In [ ]:
# classif
#decode ctxid? 
#  perturbation,  
#  prev perturbation
#      linear regr
# 
# feedback
# jax1', 'jax2'
# reward (or better nonhit)
# 
# 
# error_area2_signed_nn,    prev_error_area2_signed_nn
#      nonlin regr
#   time_lh
#  Nctx_app
#  traj_length_adj
#  traj_length
#  ES

# TODO: define angle lh and lh_ofb

In [ ]:
['current_phase_trigger', 'tgti_to_show',
       'trial_type', 'special_block_type', 
       'error_distance',
       'jax1', 'jax2',
       'reward', 'phase',
       'target_locs', 'perturbation',
       'dist_rad_from_prevtgt',
       'prev_ctx_both_close', 'prev_ctx_some_close', 'prev_ctx_tgt_close',
       'prev_ctx_pert_close', 'prev_ctx_pert_same', 'prev_ctx_tgt_same',
       'prev_block_ind_diff', 'ctxid', 'Nctx_app', 'trialwb', 'nonhit',
       'error_eucl', 'feedback_abs', 'org_feedback_abs', 'feedback',
       'org_feedback', 'error_endpoint_ang', 'error_intdist2_nn',
       'error_lh_ang', 'error_unpert_lh_ang', 'error_distance_lh',
       'error_area_signed_nn', 'error_area2_signed_nn',
       'error_area_ofb_signed_nn', 'time_lh', 'traj_length', 'enddist',
       'time_mvt', 'error_intdist2', 'error_area_signed', 'error_area2_signed',
       'error_area_ofb_signed', 'error_aug2', 'traj_length_adj',
       'error_pert_adj', 'error_endpoint_ang_pert_adj', 'error_aug3',
       'error_area_signed_tln', 'error_area_ofb_signed_tln',
       'error_area_signed_tln_scaled_ed',
       'error_area_ofb_signed_tln_scaled_ed', 'error_area_signed_scaled_ed',
       'error_area_ofb_signed_scaled_ed', 'error_area_signed_nn_scaled_ed',
       'error_area2_signed_nn_scaled_ed', 'error_area_ofb_signed_nn_scaled_ed',
       'prev_trial_type', 'prev_error_area2_signed_nn_scaled_ed',
       'prev_error_lh_ang']

In [ ]:
scores_perm.shape

In [ ]:
scores.shape

In [ ]:
res

In [ ]:
decres_naive_ITI_tgti_to_show_-2.00_6.00_(415, 274, 8).npz'

In [ ]:
fns

In [ ]:
import meg_proc

In [ ]:
phase2dat[ph]['varname2y'][vn]

In [ ]:
len(y), ph

In [ ]:
ipy.run_line_magic('run', pjoin(scripts_dir, 'td_long_cc.py'))

In [ ]:
dfev_mxph['trigger'].unique()

In [ ]:
np.unique(epochs.events[:,2] )

In [ ]:
del f
gc; collect()

In [ ]:
def f(x,a,b,c):
    a * x**2 + b * x + c
    
theta = 30 / 180 * np.pi
D = 378
f()